In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from textblob import TextBlob
from sklearn.linear_model import LinearRegression

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
df['cast'] = df['cast'].fillna('No cast specified')
cast_df = pd.DataFrame()
cast_df = df['cast'].str.split(',', expand=True).stack()
cast_df = cast_df.to_frame()
cast_df.columns = ['Actor']
actors = cast_df.groupby(['Actor']).size().reset_index(name='Total Count')
actors = actors[actors.Actor != 'No cast specified']
actors = actors.sort_values(by=['Total Count'], ascending=False)
top5Actors = actors.head()
top5Actors = top5Actors.sort_values(by=['Total Count'])
barChart2 = px.bar(top5Actors, x='Total Count', y='Actor', title='Top 5 Actors on Netflix')
barChart2.show()

df1 = df[['type', 'release_year']]
df1 = df1.rename(columns={"release_year": "Release Year", "type": "Type"})
df2 = df1.groupby(['Release Year', 'Type']).size().reset_index(name='Total Count')
df2 = df2[df2['Release Year'] >= 2000]
graph = px.line(df2, x="Release Year", y="Total Count", color="Type", title="Trend of Content Produced on Netflix Every Year")
graph.show()

df3 = df[['release_year', 'description']]
df3 = df3.rename(columns={'release_year': 'Release Year', 'description': 'Description'})
for index, row in df3.iterrows():
    d = row['Description']
    testimonial = TextBlob(d)
    p = testimonial.sentiment.polarity
    if p == 0:
        sent = 'Neutral'
    elif p > 0:
        sent = 'Positive'
    else:
        sent = 'Negative'
    df3.loc[index, 'Sentiment'] = sent

df3 = df3.groupby(['Release Year', 'Sentiment']).size().reset_index(name='Total Count')
df3 = df3[df3['Release Year'] > 2005]
barGraph = px.bar(df3, x="Release Year", y="Total Count", color="Sentiment", title="Sentiment Analysis of Content on Netflix")
barGraph.show()

In [4]:
# Prepare the dataset for prediction
df3['Release Year'] = pd.to_numeric(df3['Release Year'])
df3_pivot = df3.pivot_table(values='Total Count', index='Release Year', columns='Sentiment', fill_value=0).reset_index()

# Define the features and target variable for the prediction model
X = df3_pivot['Release Year'].values.reshape(-1, 1)
y_positive = df3_pivot['Positive'].values
y_negative = df3_pivot['Negative'].values

# Create the linear regression model for positive sentiment
model_positive = LinearRegression()
model_positive.fit(X, y_positive)

# Create the linear regression model for negative sentiment
model_negative = LinearRegression()
model_negative.fit(X, y_negative)

# Predict sentiment counts for future years (from 2023 to 2030, for example)
future_years = np.arange(2023, 2031).reshape(-1, 1)
predicted_positive = model_positive.predict(future_years)
predicted_negative = model_negative.predict(future_years)

# Create a new DataFrame to store the predictions
predictions_df = pd.DataFrame({
    'Release Year': future_years.flatten(),
    'Predicted Positive Sentiment': predicted_positive,
    'Predicted Negative Sentiment': predicted_negative
})

# Merge predictions with the original sentiment data
df3_pivot = df3_pivot.append(predictions_df, ignore_index=True)

# Plot the prediction results
fig = go.Figure()

# Actual sentiment data
fig.add_trace(go.Scatter(x=df3_pivot['Release Year'], y=df3_pivot['Positive'], mode='lines+markers', name='Positive',
                         line=dict(color='rgb(66, 191, 115)')))
fig.add_trace(go.Scatter(x=df3_pivot['Release Year'], y=df3_pivot['Negative'], mode='lines+markers', name='Negative',
                         line=dict(color='rgb(219, 64, 82)')))

# Predicted sentiment data
fig.add_trace(go.Scatter(x=predictions_df['Release Year'], y=predictions_df['Predicted Positive Sentiment'], mode='lines',
                         name='Predicted Positive',
                         line=dict(color='rgb(126, 206, 146)', dash='dash')))
fig.add_trace(go.Scatter(x=predictions_df['Release Year'], y=predictions_df['Predicted Negative Sentiment'], mode='lines',
                         name='Predicted Negative',
                         line=dict(color='rgb(242, 120, 135)', dash='dash')))

# Set layout and annotations
fig.update_layout(
    title='Sentiment Analysis of Content on Netflix (including predictions)',
    xaxis_title='Release Year',
    yaxis_title='Total Count',
    legend=dict(x=0, y=1, traceorder="normal"),
    hovermode='x unified',
    template='plotly_white',
)

# Show the figure
fig.show()

/tmp/ipykernel_19/3741761709.py:31: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

